In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Import lib**

In [ ]:
!pip install pyvi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 30.8 MB/s eta 0:00:00


In [ ]:
import torch, math
from pyvi.ViTokenizer import tokenize
import re, os, string
import pandas as pd
import math
import numpy as np

**Dataset**

In [ ]:
dataset1 = pd.read_csv('/content/drive/MyDrive/largeProject/code/MRC/dataset/visquad.csv')

In [ ]:
dataset1.drop(["Unnamed: 0"],axis = 1,inplace=True)

In [ ]:
len(dataset1)

32278

In [ ]:
df = dataset1.groupby('title')

In [ ]:
df.head()

,context,answer,question,title
0,Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4...,Lâm Bá Kiệt,Tên gọi nào được Phạm Văn Đồng sử dụng khi làm...,Phạm Văn Đồng
1,Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4...,Thủ tướng,Phạm Văn Đồng giữ chức vụ gì trong bộ máy Nhà ...,Phạm Văn Đồng
2,Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4...,Thủ tướng Chính phủ Việt Nam Dân chủ Cộng hòa,"Giai đoạn năm 1955-1976, Phạm Văn Đồng nắm giữ...",Phạm Văn Đồng
3,Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4...,Lâm Bá Kiệt,Tên gọi nào được Phạm Văn Đồng sử dụng trước k...,Phạm Văn Đồng
4,Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4...,Thủ tướng,Hồ Học Lãm giữ chức vụ gì trong bộ máy Nhà nướ...,Phạm Văn Đồng
...,...,...,...,...
32072,"Texas (phát âm: ""teksəs"" là tiểu bang đông dân...",thứ hai,Trong số 50 tiểu bang của Hoa Kỳ thì bang Texa...,Texas
32073,"Texas (phát âm: ""teksəs"" là tiểu bang đông dân...","bang Chihuahua, Coahuila, Nuevo León, và Tamau...",Những bang nào của México giáp ranh với bang T...,Texas
32074,"Texas (phát âm: ""teksəs"" là tiểu bang đông dân...",Texas,Ở phía tây nam của bang Arkansas là bang nào?,Texas
32075,"Texas (phát âm: ""teksəs"" là tiểu bang đông dân...","26,1 triệu",Dân số của bang Texas là bao nhiêu?,Texas


In [ ]:
context = {}

for ele in df:
  title,df_child = ele
  context[title] = list(set(df_child['context'].to_list()))

In [ ]:
context_full = set()

for ele in df:
  title, df_child = ele
  for child in df_child['context'].to_list():
    context_full.add(child)

context_full = list(context_full)

In [ ]:
list_stop_words = []

with open('/content/drive/MyDrive/largeProject/code/MRC/dataset/stopwords.txt','r') as f:
  for data in f.readlines():
    list_stop_words.append(data)

**Util function**

In [ ]:
def clean_text(text):
    text = re.sub('<.*?>', '', text).strip()
    text = re.sub('(\s)+', r'\1', text)
    return text

def normalize_text(text):
    listpunctuation = string.punctuation.replace('_', '')
    for i in listpunctuation:
        text = text.replace(i, ' ')
    return text.lower()

def remove_stopword(text):
    pre_text = []
    words = text.split()
    for word in words:
        if word not in list_stop_words:
            pre_text.append(word)
    text2 = ' '.join(pre_text)

    return text2

def word_segment(sent):
    sent = tokenize(sent.encode('utf-8').decode('utf-8'))
    return sent

def no_accent_vietnamese(s):
    s = re.sub(r'[àáạảãâầấậẩẫăằắặẳẵ]', 'a', s)
    s = re.sub(r'[ÀÁẠẢÃĂẰẮẶẲẴÂẦẤẬẨẪ]', 'A', s)
    s = re.sub(r'[èéẹẻẽêềếệểễ]', 'e', s)
    s = re.sub(r'[ÈÉẸẺẼÊỀẾỆỂỄ]', 'E', s)
    s = re.sub(r'[òóọỏõôồốộổỗơờớợởỡ]', 'o', s)
    s = re.sub(r'[ÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠ]', 'O', s)
    s = re.sub(r'[ìíịỉĩ]', 'i', s)
    s = re.sub(r'[ÌÍỊỈĨ]', 'I', s)
    s = re.sub(r'[ùúụủũưừứựửữ]', 'u', s)
    s = re.sub(r'[ƯỪỨỰỬỮÙÚỤỦŨ]', 'U', s)
    s = re.sub(r'[ỳýỵỷỹ]', 'y', s)
    s = re.sub(r'[ỲÝỴỶỸ]', 'Y', s)
    s = re.sub(r'[Đ]', 'D', s)
    s = re.sub(r'[đ]', 'd', s)
    return s


def convert_one_gram(docs):
    texts = [
        [word for word in document.lower().split() if word not in list_stop_words]
        for document in docs
    ]

    return texts

def split(document, k=8, max_len=400):
    document = document.replace('\n', ' ')
    document = re.sub(' +', ' ', document)
    sentences = document.split('. ')
    context_list = []

    context = ""
    length = 0
    pre = ""
    len__ = 0
    for sentence in sentences:
        sentence += '. '
        len_ = len(sentence.split())
        if length + len_ > max_len:
            context_list.append(context)
            context = pre
            length = len__

        length += len_
        context += sentence

        pre = sentence
        len__ = len_
    context_list.append(context)

    contexts = context_list
    if len(context_list) < k:
        k = len(context_list)

    return contexts,k

**Search Full Context**

In [ ]:
class BM25:
    def __init__(self, k1=1.5, b=0.75):
        self.b = b
        self.k1 = k1

    def fit(self, corpus):
        tf = []
        df = {}
        idf = {}
        doc_len = []
        corpus_size = 0
        for document in corpus:
            corpus_size += 1
            doc_len.append(len(document))

            # compute tf (term frequency) per document
            frequencies = {}
            for term in document:
                term_count = frequencies.get(term, 0) + 1
                frequencies[term] = term_count

            tf.append(frequencies)

            # compute df (document frequency) per term
            for term, _ in frequencies.items():
                df_count = df.get(term, 0) + 1
                df[term] = df_count

        for term, freq in df.items():
            idf[term] = math.log(1 + (corpus_size - freq + 0.5) / (freq + 0.5))

        self.tf_ = tf
        self.df_ = df
        self.idf_ = idf
        self.doc_len_ = doc_len
        self.corpus_ = corpus
        self.corpus_size_ = corpus_size
        self.avg_doc_len_ = sum(doc_len) / corpus_size
        return self

    def search(self, query):
        scores = [self._score(query, index) for index in range(self.corpus_size_)]
        return scores

    def _score(self, query, index):
        score = 0.0

        doc_len = self.doc_len_[index]
        frequencies = self.tf_[index]
        for term in query:
            if term not in frequencies:
                continue

            freq = frequencies[term]
            numerator = self.idf_[term] * freq * (self.k1 + 1)
            denominator = freq + self.k1 * (1 - self.b + self.b * doc_len / self.avg_doc_len_)
            score += (numerator / denominator)

        return score

In [ ]:
torch.save(BM25,'bm25.pt')

In [ ]:
torch.save(context_full,'context.pt')

**Retrieval**

In [ ]:
class Retrieval:
  def __init__(
          self, k=8,
          model='bm25.pt',
          contexts='context.pt',
          max_len = 400,
          docs = None
      ) -> None:
      self.k = k
      self.max_len = max_len

      if docs:
          self.tuning(docs)
      else:
          self.bm25 = torch.load(model)()
          self.contexts = torch.load(contexts)
          self.bm25.fit(self.contexts)

  def get_context(self, query='Chảy máu chân răng là bệnh gì?'):
      query = clean_text(query)
      query = word_segment(query)
      query = remove_stopword(normalize_text(query))
      query = query.split()

      scores = self.bm25.search(query = query)
      scores_index = np.argsort(scores)

      results = []
      ss = []
      for k in range(1, self.k+1):
          index = scores_index[-k]
          result = {'score_bm':scores[index], 'index':index, 'context':self.contexts[index]}
          results.append(result)
          ss.append(scores[index])
      print("BM25:", ss)
      return results

  def tuning(self, document):
      k, contexts = split(document,self.k,self.max_len)
      self.contexts = contexts
      self.k = k

      docs = []
      for content in self.contexts:
          content = clean_text(content)
          content = word_segment(content)
          content = remove_stopword(normalize_text(content))
          docs.append(content)

      print('There is', len(docs), 'contexts')
      texts = [
          [word for word in document.lower().split() if word not in self.list_stopwords]
          for document in docs
      ]

      self.bm25 = BM25()
      self.bm25.fit(texts)

In [ ]:
retrieve = Retrieval()

In [ ]:
retrieve.get_context('Tên gọi nào được Phạm Văn Đồng sử dụng khi làm...')

BM25: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


[{'score_bm': 0.0,
  'index': 4657,
  'context': 'Tên gọi Canada bắt nguồn từ kanata trong ngôn ngữ của người Iroquois Saint Lawrence, nghĩa là "làng" hay "khu định cư". Năm 1535, các cư dân bản địa của khu vực nay là thành phố Québec sử dụng từ này để chỉ đường cho nhà thám hiểm người Pháp Jacques Cartier đến làng Stadacona. Cartier sau đó sử dụng từ Canada để nói đến không chỉ riêng ngôi làng, mà là toàn bộ khu vực lệ thuộc vào Donnacona (tù trưởng tại Stadacona); đến khoảng năm 1545, các sách và bản đồ tại châu Âu bắt đầu gọi khu vực này là Canada.'},
 {'score_bm': 0.0,
  'index': 1528,
  'context': 'Từ quyển rất yếu bao quanh Sao Kim có nghĩa là gió Mặt Trời tương tác trực tiếp với tầng thượng quyển của hành tinh. Tại đây, các ion hiđrô và ôxy liên tục được sinh ra từ sự phân ly các phân tử trung hòa do tác động của tia tử ngoại. Tiếp đó gió Mặt Trời cung cấp năng lượng đủ lớn giúp cho những ion này có vận tốc đủ để thoát ra khỏi trường hấp dẫn của hành tinh. Sự mất mát này dẫn đến